In [1]:
import keras.backend
import keras.callbacks
import keras.layers
import keras.models
import keras.optimizers

import matplotlib
matplotlib.use('SVG')

import helper.callbacks
import helper.model_builder
import helper.visualize
#import helper.data_provider
import helper.metrics

import skimage.io
import sklearn.metrics

import scipy.stats
import pandas as pd

import tensorflow as tf
import numpy as np

import matplotlib.pyplot as plt

import sys

import os
#os.environ['CUDA_VISIBLE_DEVICES'] = ''

Using TensorFlow backend.
/home/ubuntu/.pyenv/versions/3.6.2/lib/python3.6/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [4]:
# constants
const_lr = 1e-4

tag = '01'

# general
dir_root = '/home/ubuntu/data/'

# raw data
dir_raw_images = dir_root + 'raw_images/'
dir_raw_annotations = dir_root + 'raw_annotations/'

# Split files
path_files_training = dir_root + 'unet/training.txt'
path_files_validation = dir_root + 'unet/validation.txt'
path_files_test = dir_root + 'unet/test.txt'




# Output dirs
out_dir = dir_root + 'unet/experiments/' + tag + '/out'
tb_log_dir = dir_root + "unet/tensorboard/" + tag + "/"
chkpt_dir = dir_root + "unet/experiments/" + tag + "/checkpoints/"

os.makedirs(out_dir, exist_ok=True)
os.makedirs(tb_log_dir, exist_ok=True)
os.makedirs(chkpt_dir, exist_ok=True)

# Files
chkpt_file = chkpt_dir + "{epoch:04d}.hdf5"
csv_log_file = dir_root + "unet/experiments/" + tag + "/log.csv"

# Input dirs
train_dir_x = dir_root + "unet/split/training/x/"
train_dir_y = dir_root + "unet/split/training/y/"
val_dir = dir_root + "unet/split/validation/"

# Learning Settings
rescale_labels = False

epochs = 50

batch_size = 10
# steps per epoch should be roughly the number of training images. 
# the images are big 1000x1300 compared to the size of the 
# cropped training images 256x256, i.e. 20 times smaller
# so we can choose 10* number of training images and the model 
# wont see the same image twice 
steps_per_epoch = 600# 2 * 4 / batch_size

# make sure these number for to the validation set
val_batch_size = 10
val_steps = int(50 * 4 / val_batch_size)

# generator only params
dim1 = 256
dim2 = 256

bit_depth = 8

In [6]:
# build session running on GPU 1
configuration = tf.ConfigProto()
configuration.gpu_options.allow_growth = True
configuration.gpu_options.visible_device_list = "0"
session = tf.Session(config = configuration)

# apply session
keras.backend.set_session(session)

train_gen = random_sample_generator(
    train_dir_x,
    train_dir_y,
    batch_size,
    bit_depth,
    dim1,
    dim2,
    rescale_labels
)

val_gen = single_data_from_images(
    val_dir + 'x/',
     val_dir + 'y/',
     val_batch_size,
     bit_depth,
     dim1,
     dim2,
     rescale_labels
)

Found 152 images belonging to 1 classes.
Found 152 images belonging to 1 classes.


In [7]:
# build model
model = helper.model_builder.get_model_3_class(dim1, dim2)
model.summary()

loss = "categorical_crossentropy"
metrics = [keras.metrics.categorical_accuracy, helper.metrics.recall, helper.metrics.precision]

optimizer = keras.optimizers.RMSprop(lr = const_lr)

model.compile(loss=loss, metrics=metrics, optimizer=optimizer)

# CALLBACKS
# save model after each epoch
callback_model_checkpoint = keras.callbacks.ModelCheckpoint(
    filepath=chkpt_file,
    save_weights_only=True,
    save_best_only=False
)
callback_csv = keras.callbacks.CSVLogger(filename=csv_log_file)
# callback_splits_and_merges = helper.callbacks.SplitsAndMergesLoggerBoundary(
#     'images',
#     val_gen,
#     gen_calls = val_steps,
#     log_dir=tb_log_dir
# )

callbacks=[callback_model_checkpoint, callback_csv] #, callback_splits_and_merges]


/home/ubuntu/data/unet4nuclei/unet4nuclei/helper/model_builder.py:24: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
  a = keras.layers.Convolution2D(64, 3, 3, **option_dict_conv)(x)
/home/ubuntu/data/unet4nuclei/unet4nuclei/helper/model_builder.py:27: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(momentum=0.9)`
  a = keras.layers.BatchNormalization(**option_dict_bn)(a)
/home/ubuntu/data/unet4nuclei/unet4nuclei/helper/model_builder.py:31: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
  a = keras.layers.Convolution2D(64, 3, 3, **option_dict_conv)(a)
/home/ubuntu/data/unet4nuclei/unet4nuclei/helper/model_builder.py:33: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(momentum=0.9)`
  a = keras.layers.BatchNormalization(**option_dict_bn)(a)
/home/ubuntu/data/unet4nucle

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 256, 256, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 256, 256, 64) 36928       batch_normalization_1[0][0]      
__________________________________________________________________________________________________
batch_norm

In [8]:
# TRAIN
statistics = model.fit_generator(
    generator=train_gen,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=val_gen,
    validation_steps=val_steps,
    callbacks=callbacks,
    verbose = 1
)

# visualize learning stats
#helper.visualize.visualize_learning_stats_boundary_hard(statistics, out_dir, metrics)

print('Done! :)')

Epoch 1/50
Found 304 images.
Found 304 annotations.
600/600 [==============================] - 1371s 2s/step - loss: 47.3800 - categorical_accuracy: 0.9492 - recall: 0.9999 - precision: 1.0915 - val_loss: 117.7952 - val_categorical_accuracy: 0.8821 - val_recall: 1.0000 - val_precision: 1.6216
Epoch 2/50
124/600 [=====>........................] - ETA: 17:39 - loss: 30.9007 - categorical_accuracy: 0.9600 - recall: 0.9999 - precision: 1.0357

KeyboardInterrupt: 

In [3]:
import numpy as np
import keras.preprocessing.image

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

import skimage.io


def data_from_array(data_dir):
    
    # load  x
    training_x = np.load(data_dir+"training/x.npy")
    test_x = np.load(data_dir+"test/x.npy")
    validation_x = np.load(data_dir+"validation/x.npy")

    print(training_x.shape)
    print(test_x.shape)
    print(validation_x.shape)

    # normalize
    training_x = training_x / 255
    test_x = test_x / 255
    validation_x = validation_x / 255

    # load y
    training_y = np.load(data_dir+"training/y.npy")
    test_y = np.load(data_dir+"test/y.npy")
    validation_y = np.load(data_dir+"validation/y.npy")

    print(training_y.shape)
    print(test_y.shape)
    print(validation_y.shape)
    
    return [training_x, training_y, validation_x, validation_y, test_x, test_y]


def data_from_images(data_dir, batch_size, bit_depth, dim1, dim2):
    
    flow_train = single_data_from_images(data_dir + 'training/x/', data_dir + 'training/y/', batch_size, bit_depth, dim1, dim2)
    flow_validation = single_data_from_images(data_dir + 'validation/x', data_dir + 'validation/y', batch_size, bit_depth, dim1, dim2)
    flow_test = single_data_from_images(data_dir + 'test/x', data_dir + 'test/y', batch_size, bit_depth, dim1, dim2)
    
    return [flow_train, flow_validation, flow_test]


def single_data_from_images(x_dir, y_dir, batch_size, bit_depth, dim1, dim2, rescale_labels):

    rescale_factor = 1./(2**bit_depth - 1)
    
    if(rescale_labels):
        rescale_factor_labels = rescale_factor
    else:
        rescale_factor_labels = 1

    gen_x = keras.preprocessing.image.ImageDataGenerator(rescale=rescale_factor)
    gen_y = keras.preprocessing.image.ImageDataGenerator(rescale=rescale_factor_labels)
    
    seed = 42

    stream_x = gen_x.flow_from_directory(
        x_dir,
        target_size=(dim1,dim2),
        color_mode='grayscale',
        batch_size=batch_size,
        class_mode=None,
        seed=seed
    )
    stream_y = gen_y.flow_from_directory(
        y_dir,
        target_size=(dim1,dim2),
        color_mode='rgb',
        batch_size=batch_size,
        class_mode=None,
        seed=seed
    )
    
    flow = zip(stream_x, stream_y)
    
    return flow


def single_data_from_images_1d_y(x_dir, y_dir, batch_size, bit_depth, dim1, dim2, rescale_labels):

    rescale_factor = 1./(2**bit_depth - 1)
    
    if(rescale_labels):
        rescale_factor_labels = rescale_factor
    else:
        rescale_factor_labels = 1

    gen_x = keras.preprocessing.image.ImageDataGenerator(rescale=rescale_factor)
    gen_y = keras.preprocessing.image.ImageDataGenerator(rescale=rescale_factor_labels)
    
    seed = 42

    stream_x = gen_x.flow_from_directory(
        x_dir,
        target_size=(dim1,dim2),
        color_mode='grayscale',
        batch_size=batch_size,
        class_mode=None,
        seed=seed
    )
    stream_y = gen_y.flow_from_directory(
        y_dir,
        target_size=(dim1,dim2),
        color_mode='grayscale',
        batch_size=batch_size,
        class_mode=None,
        seed=seed
    )
    
    flow = zip(stream_x, stream_y)
    
    return flow


def random_sample_generator(x_big_dir, y_big_dir, batch_size, bit_depth, dim1, dim2, rescale_labels):

    debug = False
    do_augmentation = True
    
    # get images
    x_big = skimage.io.imread_collection(x_big_dir + '*.png').concatenate()
    print('Found',len(x_big), 'images.')
    y_big = skimage.io.imread_collection(y_big_dir + '*.png').concatenate()
    print('Found',len(y_big), 'annotations.')
    
    if(len(y_big.shape) == 3):
        gray = True
    else:
        gray = False
    
    if(debug):
        fig = plt.figure()
        plt.hist(y_big.flatten())
        plt.savefig('y_hist')
        plt.close(fig)

        fig = plt.figure()
        plt.hist(x_big.flatten())
        plt.savefig('x_hist')
        plt.close(fig)
    
    # get dimensions right -- understand data set
    n_images = x_big.shape[0]
    dim1_size = x_big.shape[1]
    dim2_size = x_big.shape[2]
    
    # rescale images
    rescale_factor = 1./(2**bit_depth - 1)
    if(rescale_labels):
        rescale_factor_labels = rescale_factor
    else:
        rescale_factor_labels = 1
        
    while(True):
        
        if(gray):
            y_channels = 1
        else:
            y_channels = 3
            
        # buffers for a batch of data
        x = np.zeros((batch_size, dim1, dim2, 1))
        y = np.zeros((batch_size, dim1, dim2, y_channels))
        
        # get one image at a time
        for i in range(batch_size):
                       
            # get random image
            img_index = np.random.randint(low=0, high=n_images)
            
            # get random crop
            start_dim1 = np.random.randint(low=0, high=dim1_size-dim1)
            start_dim2 = np.random.randint(low=0, high=dim2_size-dim2)
            
            patch_x = x_big[img_index, start_dim1:start_dim1 + dim1, start_dim2:start_dim2 + dim2] * rescale_factor
            patch_y = y_big[img_index, start_dim1:start_dim1 + dim1, start_dim2:start_dim2 + dim2] * rescale_factor_labels
            
            if(do_augmentation):
                
                rand_flip = np.random.randint(low=0, high=2)
                rand_rotate = np.random.randint(low=0, high=4)
                
                # flip
                if(rand_flip == 0):
                    patch_x = np.flip(patch_x, 0)
                    patch_y = np.flip(patch_y, 0)
                
                # rotate
                for rotate_index in range(rand_rotate):
                    patch_x = np.rot90(patch_x)
                    patch_y = np.rot90(patch_y)
                    
            # save image to buffer
            x[i, :, :, 0] = patch_x
            
            if(gray):
                y[i, :, :, 0] = patch_y
            else:
                y[i, :, :, 0:y_channels] = patch_y
            
            if(debug):
                fig = plt.figure()
                plt.imshow(x[i, :, :, 0])
                plt.colorbar()
                plt.savefig('/home/jr0th/github/segmentation/code/generated/x_' + str(i))
                plt.close(fig)

                fig = plt.figure()
                plt.imshow(y[i, :, :, 0])
                plt.colorbar()
                plt.savefig('/home/jr0th/github/segmentation/code/generated/y_' + str(i))
                plt.close(fig)
            
        # return the buffer
        yield(x, y)


/home/ubuntu/.pyenv/versions/3.6.2/lib/python3.6/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)
